In [4]:
!pip3 install catboost

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
import catboost as cb

data = pd.read_csv("Metabolic Syndrome.csv",
                   usecols=["Age", "WaistCirc", "BMI", "MetabolicSyndrome"])
data.dropna(inplace=True)
print(data.head())
print(data.isnull().sum())

predicts = data[['Age', 'WaistCirc', 'BMI']]
target = data['MetabolicSyndrome']
print(predicts)
print(target)
print(predicts.shape)
#Разделение на обучающую и тестовую выборки
A_train, A_test, y_train, y_test = train_test_split(predicts,target,train_size = 0.8)
print(A_train.shape)
print(A_test.shape)
#Обучение случайного леса с остановой
random_forest = RandomForestClassifier(max_depth=15,min_samples_split=10).fit(A_train, y_train)
y_preds_d = random_forest.predict(A_train)
print('F1 мера для тренировочных данных ', f1_score(y_preds_d,y_train,average='macro'))

y_pred = random_forest.predict(A_test)
print('F1 мера для тестовых данных ', f1_score(y_pred,y_test,average='macro'))

random_forest = RandomForestClassifier()

params_grid = {
    "max_depth": [12,18],
    "min_samples_leaf": [3,10],
    "min_samples_split": [6,12], #минимум примеров в вершине, при котором можно продолжить деление
}

grid_search_random_forest = GridSearchCV(estimator=random_forest,
                                         param_grid=params_grid,
                                         scoring="f1_macro",
                                         cv = 4)

grid_search_random_forest.fit(A_train, y_train)
best_model = grid_search_random_forest.best_estimator_
y_preds_d = best_model.predict(A_train)
print('F1 мера для тренировочных данных ', f1_score(y_preds_d,y_train,average='macro'))

y_pred = best_model.predict(A_test)
print('F1 мера для тестовых данных ', f1_score(y_pred,y_test,average="macro"))

model_catboost_clf = cb.CatBoostClassifier(iterations=3000,
                                           task_type="GPU",
                                           devices='0')
model_catboost_clf.fit(A_train, y_train)
y_preds_t = model_catboost_clf.predict(A_train,task_type="CPU")
print('F1 мера для тренировочных данных ', f1_score(y_preds_t,y_train,average='macro'))

y_preds = model_catboost_clf.predict(A_test,task_type="CPU")
print('F1 мера для тестовых данных ', f1_score(y_preds,y_test,average="macro"))

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/e2/63/379617e3d982e8a66c9d66ebf4621d3357c7c18ad356473c335bffd5aba6/catboost-1.2.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     -------------------------------------- 47.0/47.0 kB 782.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/101.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/101.0 MB 3.1 MB/s eta 0:00:34
   ---------------------------------------- 0.3/101.0 MB 3.5 MB/s eta 0:00:29
   ---------------------------------------- 0.6/101.0 MB 3.8 MB/s eta 0:00:27
   ---------------------------------------- 0.6/101.0 MB 3.1 MB/s eta 0:00:33
   ---------------------------------------- 0.9/101.0 MB 3.9 MB/s eta 0:00:26
   ---------------------------------------- 1.0/101.0 MB 4.1 MB/s eta 0:00:25
    --------------------------------------- 1.6/101.0 MB 5.0 MB/s eta 0:00:21
    -----------------

   Age  WaistCirc   BMI  MetabolicSyndrome
0   22       81.0  23.3                  0
1   44       80.1  23.2                  0
2   21       69.6  20.1                  0
3   43      120.4  33.3                  0
4   51       81.1  20.1                  0
Age                  0
WaistCirc            0
BMI                  0
MetabolicSyndrome    0
dtype: int64
      Age  WaistCirc   BMI
0      22       81.0  23.3
1      44       80.1  23.2
2      21       69.6  20.1
3      43      120.4  33.3
4      51       81.1  20.1
...   ...        ...   ...
2394   31       74.0  20.6
2395   65       98.5  29.4
2398   28      100.8  29.4
2399   27      106.6  31.3
2400   60      106.6  27.5

[2311 rows x 3 columns]
0       0
1       0
2       0
3       0
4       0
       ..
2394    0
2395    1
2398    0
2399    1
2400    1
Name: MetabolicSyndrome, Length: 2311, dtype: int64
(2311, 3)
(1848, 3)
(463, 3)
F1 мера для тренировочных данных  0.8766273609670887
F1 мера для тестовых данных  0.7007616433997

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version